In [1]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

from tensorflow import keras


rng = 69

In [2]:
#df = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Transformed.csv")
df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_All_Transformed.csv")

In [3]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away'])

In [4]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [5]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [6]:
corr_matrix['win_or_lose_Home'].sort_values(ascending=False)

win_or_lose_Home                                 1.000000
seasonPointsPerGame_Home                         0.302867
fenwickPercentageAvg_Home                        0.156888
scoreAdjustedUnblockedShotAttemptsForAvg_Home    0.151217
xGoalsPercentageAvg_Home                         0.150541
                                                   ...   
scoreAdjustedUnblockedShotAttemptsForAvg_Away   -0.141069
corsiPercentageAvg_Away                         -0.156908
xGoalsPercentageAvg_Away                        -0.159364
fenwickPercentageAvg_Away                       -0.160768
seasonPointsPerGame_Away                        -0.294493
Name: win_or_lose_Home, Length: 203, dtype: float64

In [7]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [8]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [9]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('win_or_lose_Home', axis=1)
nhl_labels = train_set['win_or_lose_Home'].copy()

X_train = train_set.drop('win_or_lose_Home', axis=1)
Y_train = train_set['win_or_lose_Home'].copy()

X_valid = train_set.drop('win_or_lose_Home', axis=1)
Y_valid = train_set['win_or_lose_Home'].copy()

X_test = test_set.drop('win_or_lose_Home', axis=1)
Y_test = test_set['win_or_lose_Home'].copy()

df_Final = df_Final.drop('win_or_lose_Home', axis=1)

In [10]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [11]:
# Fine Tune the hyperparameters of RandomForest model
# from sklearn.model_selection import GridSearchCV

# param_grid = [{'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},
#               {'bootstrap': [False], 'n_estimators' : [100,500,1000], 'max_leaf_nodes': [10,16,25,50], 'n_jobs' : [-1]},]

# rnd_clf_test = RandomForestClassifier()

# grid_search = GridSearchCV(rnd_clf_test, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

# grid_search.fit(nhl_prepared,nhl_labels)

# grid_search.best_params_

In [12]:
# cvres = grid_search.cv_results_
# results = []
# for mean_scores, params in zip(cvres["mean_test_score"], cvres["params"]):
#     results.append([np.sqrt(-mean_scores),params])

# results2 = sorted(results, key=lambda x: x[0])
# print(*results2, sep = '\n')

In [13]:
# # baseline model
# from tensorflow import keras


# # create model
# base_model = keras.models.Sequential([
#     keras.Input(shape=(X_train.shape[1],)),
#     keras.layers.Dense(30, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
#                        ])
    
# # Compile model
# base_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [14]:
def build_model(n_hidden=1, n_neurons=50, learning_rate=1e-3, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='selu',kernel_initializer="lecun_normal"))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=optimizer)
    return model

In [15]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=build_model, n_hidden=1, n_neurons=30, learning_rate=3e-3)


In [16]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'model__n_hidden': [0,1,2,3],
    'model__n_neurons': np.arange(1,50),
    'model__learning_rate' : reciprocal(1e-5, 1e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_clf, param_distribs, n_iter=10, cv=3, error_score='raise')

early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
rnd_search_cv.fit(X_train_prepared,Y_train, epochs=100,
                 validation_data = (X_valid_prepared, Y_valid),
                 callbacks=[early_stopping_cb])

Epoch 1/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.9671 - val_loss: 0.8830
Epoch 2/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8543 - val_loss: 0.8247
Epoch 3/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8127 - val_loss: 0.7908
Epoch 4/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7660 - val_loss: 0.7695
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7496 - val_loss: 0.7545
Epoch 6/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7430 - val_loss: 0.7421
Epoch 7/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7135 - val_loss: 0.7322
Epoch 8/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7183 - val_loss: 0.7235
Epoch 9/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7242 - val_loss: 0.7160
Epoch 10/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6868 - val_loss: 0.7094
Epoch 11/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6890 - val_loss: 0.7037
Epoch 12/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5638 - val_loss: 0.5941
Epoch 70/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5597 - val_loss: 0.5933
Epoch 71/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5739 - val_loss: 0.5925
Epoch 72/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5615 - val_loss: 0.5918
Epoch 73/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5595 - val_loss: 0.5910
Epoch 74/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5666 - val_loss: 0.5903
Epoch 75/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5533 - val_loss: 0.5897
Epoch 76/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5686 - val_loss: 0.5892
Epoch 77/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5583 - val_loss: 0.5885
Epoch 78/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5693 - val_loss: 0.5879
Epoch 79/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5671 - val_loss: 0.5873
Epoch 80/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5715 

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6072 - val_loss: 0.6189
Epoch 37/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6149 - val_loss: 0.6179
Epoch 38/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6263 - val_loss: 0.6168
Epoch 39/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6023 - val_loss: 0.6158
Epoch 40/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6079 - val_loss: 0.6146
Epoch 41/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5997 - val_loss: 0.6136
Epoch 42/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6147 - val_loss: 0.6126
Epoch 43/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6103 - val_loss: 0.6116
Epoch 44/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6017 - val_loss: 0.6106
Epoch 45/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6098 - val_loss: 0.6096
Epoch 46/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6004 - val_loss: 0.6087
Epoch 47/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5965 

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7003 - val_loss: 0.6983
Epoch 4/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6950 - val_loss: 0.6860
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6737 - val_loss: 0.6780
Epoch 6/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6648 - val_loss: 0.6719
Epoch 7/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6725 - val_loss: 0.6667
Epoch 8/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6649 - val_loss: 0.6627
Epoch 9/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6536 - val_loss: 0.6592
Epoch 10/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6490 - val_loss: 0.6562
Epoch 11/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6588 - val_loss: 0.6533
Epoch 12/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6498 - val_loss: 0.6507
Epoch 13/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6357 - val_loss: 0.6481
Epoch 14/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6435 - val_

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5689 - val_loss: 0.5843
Epoch 72/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5570 - val_loss: 0.5837
Epoch 73/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5651 - val_loss: 0.5831
Epoch 74/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5779 - val_loss: 0.5828
Epoch 75/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5640 - val_loss: 0.5822
Epoch 76/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5772 - val_loss: 0.5820
Epoch 77/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5589 - val_loss: 0.5813
Epoch 78/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5671 - val_loss: 0.5812
Epoch 79/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5497 - val_loss: 0.5806
Epoch 80/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5628 - val_loss: 0.5804
Epoch 81/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5574 - val_loss: 0.5800
Epoch 82/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5634 

...

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4538 - val_loss: 0.5345
Epoch 35/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4492 - val_loss: 0.5425
Epoch 36/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4372 - val_loss: 0.5349
Epoch 37/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4463 - val_loss: 0.5212
Epoch 38/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4304 - val_loss: 0.5400
Epoch 39/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4492 - val_loss: 0.5571
Epoch 40/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4385 - val_loss: 0.5271
Epoch 41/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4187 - val_loss: 0.5211
Epoch 42/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4368 - val_loss: 0.5233
Epoch 43/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4133 - val_loss: 0.5201
Epoch 44/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4041 - val_loss: 0.5331
Epoch 45/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4299 

69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.7355 - val_loss: 0.6204
Epoch 2/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6137 - val_loss: 0.6066
Epoch 3/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6021 - val_loss: 0.5920
Epoch 4/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6012 - val_loss: 0.5929
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5787 - val_loss: 0.5802
Epoch 6/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5538 - val_loss: 0.5856
Epoch 7/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5772 - val_loss: 0.5760
Epoch 8/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5711 - val_loss: 0.5741
Epoch 9/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5551 - val_loss: 0.5684
Epoch 10/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5347 - val_loss: 0.5698
Epoch 11/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5458 - val_loss: 0.5628
Epoch 12/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5251 - val_lo

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3818 - val_loss: 0.5336
Epoch 70/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3678 - val_loss: 0.5427
Epoch 71/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3629 - val_loss: 0.5401
Epoch 72/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3499 - val_loss: 0.5614
Epoch 73/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3619 - val_loss: 0.5307
Epoch 74/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3658 - val_loss: 0.5412
Epoch 75/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3617 - val_loss: 0.5403
Epoch 76/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3585 - val_loss: 0.5459
Epoch 77/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3411 - val_loss: 0.5425
Epoch 78/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3453 - val_loss: 0.5361
Epoch 79/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3603 - val_loss: 0.5464
Epoch 80/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3693 

103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5886 - val_loss: 0.5860
Epoch 37/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5862 - val_loss: 0.5857
Epoch 38/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5739 - val_loss: 0.5850
Epoch 39/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5898 - val_loss: 0.5845
Epoch 40/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5790 - val_loss: 0.5844
Epoch 41/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5970 - val_loss: 0.5839
Epoch 42/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5825 - val_loss: 0.5838
Epoch 43/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5837 - val_loss: 0.5832
Epoch 44/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5779 - val_loss: 0.5830
Epoch 45/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5944 - val_loss: 0.5827
Epoch 46/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5804 - val_loss: 0.5824
Epoch 47/100
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 

RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=KerasClassifier(learning_rate=0.003, model=<function build_model at 0x0000028321F4DBC0>, n_hidden=1, n_neurons=30),
                   param_distributions={'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000028321F52550>,
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])})

In [17]:
#print(rnd_search_cv.best_params_)
#print(rnd_search_cv.best_score_)

In [18]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)

{'model__learning_rate': 0.0003341554191938717, 'model__n_hidden': 0, 'model__n_neurons': 46}
0.663436926985011


In [19]:
model = rnd_search_cv.best_estimator_.model

In [20]:
from scikeras.wrappers import KerasClassifier

# Seems like you need to specify some items again when using the scikeras wrapper
keras_clf = KerasClassifier(model=model, epochs=100, batch_size=5, verbose=0, random_state = 42,
                           loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

# Stop early if model is not getting better after # of patience epochs, restore to best model
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
#checkpoint_cb = keras.callbacks.ModelCheckpoint("Regression_Sequential.keras", save_best_only = True)

keras_clf.fit(X_train_prepared, Y_train, validation_data = (X_valid_prepared, Y_valid),
             callbacks=[early_stopping_cb])



KerasClassifier(
	model=<function build_model at 0x0000028321F4DBC0>
	build_fn=None
	warm_start=False
	random_state=42
	optimizer=adam
	loss=binary_crossentropy
	metrics=accuracy
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
	class_weight=None
)

In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

#kfold = StratifiedKFold(n_splits=5, shuffle=True)
#results = cross_val_score(keras_clf, X_train_prepared, Y_train, cv=kfold)

#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [22]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
Y_pred = cross_val_predict(keras_clf, X_train_prepared, Y_train, cv=kfold)

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_pred)
# [Predict Loss - Actual Loss  |  Predict Win - Actual Loss]
# [Predict Loss - Actual Win   |  Predict Win - Actual Win]

array([[ 831,  695],
       [ 662, 1107]], dtype=int64)

In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

print('When claiming is a win, correct % is ', precision_score(Y_train, Y_pred))
print('Detected % of wins' ,recall_score(Y_train, Y_pred))
print('F1 score is ', f1_score(Y_train, Y_pred))
print('Training Accuracy score is ', accuracy_score(Y_train, Y_pred))


When claiming is a win, correct % is  0.6143174250832408
Detected % of wins 0.6257772752967778
F1 score is  0.6199943993279193
Training Accuracy score is  0.5881638846737481


In [24]:
# Train Accuracy
Y_train_pred = keras_clf.predict(X_train_prepared)
print(accuracy_score(Y_train, Y_train_pred))

Y_test_pred = keras_clf.predict(X_test_prepared)
print(accuracy_score(Y_test, Y_test_pred))

0.9195751138088012
0.5752427184466019


In [25]:
#df_Predict = pd.read_csv(r"C:\Users\zchodaniecky\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Predict.csv")
df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Predict.csv")

In [26]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_clf.predict(X_real_predict))).T

ValueError: Found array with 0 sample(s) (shape=(0, 14)) while a minimum of 1 is required.